In [2]:
# STEP 0: IMPORTS (Like ML libraries: pandas, sklearn)
from langchain_community.document_loaders import TextLoader  # Loads text docs (like pd.read_csv)
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Splits docs into chunks (like train_test_split)
from langchain_community.vectorstores import FAISS  # Vector "model" store (like sklearn's fit/transform)
from langchain_community.embeddings import HuggingFaceEmbeddings  # Vectorizer (like StandardScaler)
from langchain_core.prompts import ChatPromptTemplate  # Prompt "template" (like model formula)
from langchain_core.output_parsers import StrOutputParser  # "Predict" cleaner (like predict_proba → label)
from langchain_core.runnables import RunnablePassthrough  # Passes data through (like X = X, y = y)
from langchain_ollama import ChatOllama  # LLM "model" (like LinearRegression)
from langchain_core.chat_history import InMemoryChatMessageHistory  # Memory "store" (like session df)
from langchain_core.runnables import RunnableWithMessageHistory  # Memory wrapper (like fit with history)
from langchain_core.tools import tool  # Tool def (like custom feature)
from langchain_core.messages import AIMessage, HumanMessage  # Message types (like data rows)
import redis  # Caching "store" (like joblib dump)
import json  # Serialize for Redis (like pickle)

In [ ]:
# LLM "model" (like model = LinearRegression())
llm = ChatOllama(model="mistral", temperature=0.5)

# Redis for caching (like cache = joblib.Memory(location='cache'))
r = redis.Redis(host='localhost', port=6379, db=0)

# STEP 1: DATA LOADING & EDA (Like pd.read_csv + df.describe())
print("=== STEP 1: DATA LOADING & EDA ===")
loader = TextLoader("E:\LTI\ollama-env\lc-proj\sample_docs\gdpr_guideline.txt")  # What: Loads full doc as Document (text + metadata). Why: Raw input for RAG (like df = pd.read_csv). How: Like reading file.
docs = loader.load()  # What: List of Document objects. Why: Standard LangChain format. How: Connects to splitter.

print(f"Loaded {len(docs)} docs.")
print("Sample content:", docs[0].page_content[:200])  # EDA: Preview (like df.head())

In [ ]:
# STEP 2: DATA PREP - CHUNKING (Like train_test_split + scaling)
print("\n=== STEP 2: DATA PREP - CHUNKING ===")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)  # What: Splits doc into overlapping chunks. Why: Docs too long for LLM/embed (token limit); overlap preserves context (like CV split). How: Recursive (sentences > words > chars).
splits = splitter.split_documents(docs)  # What: List of smaller Documents. Why: Embeddable units. How: Input to embeddings (like X_train = scaler.fit_transform(X)).

print(f"Created {len(splits)} chunks.")  # EDA: Check split (like len(X_train))
for i, chunk in enumerate(splits[:2]):
    print(f"Chunk {i+1} (len {len(chunk.page_content)} chars): {chunk.page_content[:100]}...")

In [6]:
# STEP 3: "MODEL BUILD" - EMBEDDING & INDEXING (Like fit(transform))
print("\n=== STEP 3: MODEL BUILD - EMBEDDING & INDEXING ===")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # What: Text → vectors (384-dim). Why: Semantic search (similar texts close in vector space). How: Pre-trained model (like fit on chunks).
vectorstore = FAISS.from_documents(splits, embeddings)  # What: Builds FAISS index (vector store). Why: Fast similarity search (cosine). How: Embeds chunks, stores for retrieval (like model.fit(X)).

retriever = vectorstore.as_retriever(k=3)  # What: Wrapper for top-k search. Why: Easy LCEL plug-in. How: query → embed → top matches (like predict(X_test)).

print("Index built with {len(splits)} vectors.")  # EDA: Verify (like model.coef_)


=== STEP 3: MODEL BUILD - EMBEDDING & INDEXING ===


C:\Users\admin\AppData\Local\Temp\ipykernel_11768\590570736.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # What: Text → vectors (384-dim). Why: Semantic search (similar texts close in vector space). How: Pre-trained model (like fit on chunks).


Index built with {len(splits)} vectors.


In [ ]:
help(ChatPromptTemplate.from_messages)

In [ ]:
# STEP 4: "PREDICT" - RETRIEVE + AUGMENT + GENERATE (Like model.predict)
print("\n=== STEP 4: PREDICT - RETRIEVE + AUGMENT + GENERATE ===")
prompt = ChatPromptTemplate.from_template("FAQ Bot: Context: {context}\nQuestion: {question}\nAnswer:")  # What: Prompt template. Why: Augments with retrieved context (RAG core). How: {context} = chunks, {question} = input.
parser = StrOutputParser()  # What: Strips AIMessage to string. Why: Clean output. How: Post-LLM (like to_label).

chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # What: Dict runnable. Why: Retrieve chunks as 'context', pass question. How: Parallel (retrieve + query).
    | prompt  # Augment: Stuff context into template.
    | llm  # Generate: LLM reasons over augmented prompt.
    | parser  # Clean: String output.
)

query = "gdpr rules in eu?"  # Test input (like X_test)
response = chain.invoke(query)  # What: Runs pipeline. Why: End-to-end predict. How: LCEL pipe executes steps.
print("RAG Response:", response)  # Output: Grounded answer (e.g., "Up to $50k for breaches...").

In [ ]:
# STEP 5: INTEGRATE MEMORY (Retain user history for recurring issues)
print("\n=== STEP 5: INTEGRATE MEMORY ===")
store = {}  # What: Session dict. Why: Per-user history. How: Key = session_id.
def get_session_history(session_id: str):  # What: Loader. Why: Scopes memory. How: InMemory for messages.
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

memory_chain = RunnableWithMessageHistory(  # What: Wrapper. Why: Auto-loads/appends history. How: Wraps chain.
    chain,
    get_session_history,
    input_messages_key="question",  # Maps query to HumanMessage.
    history_messages_key="context"  # Injects history as context (augment with past).
)

config = {"configurable": {"session_id": "user1"}}  # What: Session. Why: Scopes. How: Passed to invoke.
response1 = memory_chain.invoke({"question": "Outlook basics?"}, config)  # Turn 1
print("Turn 1:", response1)

response2 = memory_chain.invoke({"question": "More on crashes from basics?"}, config)  # Turn 2
print("Turn 2 (memory):", response2)  # Output: References "basics" from turn 1.

In [ ]:
# STEP 6: INTEGRATE TOOLS (Enrich with external action)
print("\n=== STEP 6: INTEGRATE TOOLS ===")
@tool  # What: Decorator. Why: LLM-callable. How: Schema from docstring.
def kb_lookup(query: str) -> str:  # What: Tool. Why: Mock KB enrich. How: LLM binds.
    """Mock IT KB lookup for fixes."""
    return f"KB for '{query}': Restart service or update patch."

llm_with_tools = llm.bind_tools([kb_lookup])  # What: Bind. Why: LLM decides call. How: Outputs tool_calls.

tool_chain = (
    {"context": hybrid_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm_with_tools  # Tool-enabled LLM.
    | StrOutputParser()
)

response_tool = tool_chain.invoke("Outlook crash? Fix?")  # What: Invoke. Why: Triggers if relevant. How: LLM reasons.
print("With Tool:", response_tool)  # Output: RAG + "KB: Restart..."


In [ ]:
# Manual resolve (like post-process)
if hasattr(response_tool, 'tool_calls') and response_tool.tool_calls:
    tool_res = kb_lookup.invoke(response_tool.tool_calls[0]['args'])
    response_tool += f"\nTool KB: {tool_res}"  # Augment.
print("Resolved:", response_tool)

# STEP 7: PERSISTENCE - SAVE INDEX (Like joblib.dump)
print("\n=== STEP 7: PERSISTENCE - SAVE INDEX ===")
vectorstore.save_local("ticket_db")  # What: Disk save. Why: Survives restarts. How: FAISS folder (index.faiss + index.pkl).

# STEP 8: CACHING WITH REDIS (Optional - Like joblib.Memory)
print("\n=== STEP 8: CACHING WITH REDIS (Optional) ===")
cache_key = f"rag:{query}"  # What: Key. Why: Cache hits fast. How: Query as key.
cached = r.get(cache_key)  # Get.
if cached:
    print("Cache Hit:", cached.decode())
else:
    response = chain.invoke(query)  # Miss: Run chain.
    r.setex(cache_key, 3600, response)  # Set with 1h TTL.
    print("Cache Miss - Stored:", response)

# STEP 9: EVALUATE (Like model.score)
print("\n=== STEP 9: EVALUATE ===")
retrieved = hybrid_retriever.invoke(query)  # Contexts.
mock_ground = "Outlook crash: Restart service."  # Mock truth.
score = 1 if "restart" in response.lower() else 0  # Mock relevance.
print(f"Eval Score: {score}/1 - {'Pass' if score == 1 else 'Fail'}")

# Full test with memory/tool/cache
print("\n=== FULL TEST ===")
config = {"configurable": {"session_id": "user2"}}
response_full = memory_chain.invoke({"question": "Outlook basics? Crashes?"}, config)
print("Full Pipeline:", response_full)